## Importing libraries

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
matplotlib.rc('xtick', labelsize=20) 
matplotlib.rc('ytick', labelsize=20)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import f1_score, log_loss
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process.kernels import RBF
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost.sklearn import XGBClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import KFold

## Reading the data from drive 

In [0]:
# Import PyDrive and associated libraries.
# This only needs to be done once in a notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)



In [0]:
link = 'https://drive.google.com/open?id=1VE5uiRpHlkavElYpy2AvEm1r-iY7PWMa' # The shareable link

In [6]:
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='

1VE5uiRpHlkavElYpy2AvEm1r-iY7PWMa


### Train data

In [0]:
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('train.csv')  
train_df = pd.read_csv('train.csv')
# Dataset is now stored in a Pandas Dataframe

In [8]:
train_df.head()

,id,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,feat_10,feat_11,feat_12,feat_13,feat_14,feat_15,feat_16,feat_17,feat_18,feat_19,feat_20,feat_21,feat_22,feat_23,feat_24,feat_25,feat_26,feat_27,feat_28,feat_29,feat_30,feat_31,feat_32,feat_33,feat_34,feat_35,feat_36,feat_37,feat_38,feat_39,...,feat_55,feat_56,feat_57,feat_58,feat_59,feat_60,feat_61,feat_62,feat_63,feat_64,feat_65,feat_66,feat_67,feat_68,feat_69,feat_70,feat_71,feat_72,feat_73,feat_74,feat_75,feat_76,feat_77,feat_78,feat_79,feat_80,feat_81,feat_82,feat_83,feat_84,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93,target
0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,2,0,0,0,0,1,0,4,1,1,0,0,2,0,0,0,0,0,1,0,0,0,0,...,0,0,2,0,0,11,0,1,1,0,1,0,7,0,0,0,1,0,0,0,0,0,0,0,2,1,0,0,0,0,1,0,0,0,0,0,0,0,0,Class_1
1,2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,2,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Class_1
2,3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,6,0,0,2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,Class_1
3,4,1,0,0,1,6,1,5,0,0,1,1,0,1,0,0,1,1,0,0,0,0,0,0,7,2,2,0,0,0,58,0,10,0,0,0,0,0,3,0,...,1,0,0,0,0,0,0,0,0,0,2,1,5,0,0,4,0,0,2,1,0,1,0,0,1,1,2,2,0,22,0,1,2,0,0,0,0,0,0,Class_1
4,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,3,0,0,0,0,0,0,0,4,0,1,0,0,0,1,0,0,0,0,1,0,0,0,Class_1


In [0]:
link = 'https://drive.google.com/open?id=1TAS5yTMW0VJVhNz1VJy1f5pcc791GkV6' # The shareable link

In [10]:
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='

1TAS5yTMW0VJVhNz1VJy1f5pcc791GkV6


### Test Data

In [11]:
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('test.csv')  
test_df = pd.read_csv('test.csv')
# Dataset is now stored in a Pandas Dataframe
test_df.head()

,id,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,feat_10,feat_11,feat_12,feat_13,feat_14,feat_15,feat_16,feat_17,feat_18,feat_19,feat_20,feat_21,feat_22,feat_23,feat_24,feat_25,feat_26,feat_27,feat_28,feat_29,feat_30,feat_31,feat_32,feat_33,feat_34,feat_35,feat_36,feat_37,feat_38,feat_39,...,feat_54,feat_55,feat_56,feat_57,feat_58,feat_59,feat_60,feat_61,feat_62,feat_63,feat_64,feat_65,feat_66,feat_67,feat_68,feat_69,feat_70,feat_71,feat_72,feat_73,feat_74,feat_75,feat_76,feat_77,feat_78,feat_79,feat_80,feat_81,feat_82,feat_83,feat_84,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93
0,1,0,0,0,0,0,0,0,0,0,3,0,0,0,3,2,1,0,0,0,0,0,0,0,5,3,1,1,0,0,0,0,0,1,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,11,1,20,0,0,0,0,0
1,2,2,2,14,16,0,0,0,0,0,0,0,0,0,0,2,2,0,0,0,0,0,2,0,4,0,4,0,0,0,0,2,0,0,0,8,0,0,0,0,...,24,0,0,0,0,0,0,0,0,0,6,8,0,0,0,0,0,0,0,0,0,0,0,2,0,0,4,0,2,0,0,0,0,0,0,4,0,0,2,0
2,3,0,1,12,1,0,0,0,0,0,0,7,1,0,0,0,7,0,2,0,0,0,4,0,0,0,1,1,2,0,0,0,0,0,0,1,0,0,2,0,...,10,1,0,2,0,0,1,6,1,1,0,0,1,1,1,2,0,0,2,0,0,0,0,0,0,0,6,0,2,0,0,0,0,0,2,0,0,0,0,1
3,4,0,0,0,1,0,0,0,0,0,0,0,0,0,0,21,3,0,0,0,0,0,0,0,0,4,0,0,0,1,0,0,0,2,0,0,0,0,1,0,...,2,0,0,0,0,0,0,0,9,0,0,2,0,0,0,0,6,0,8,0,0,0,0,1,0,0,0,0,0,0,0,3,1,0,0,0,0,0,0,0
4,5,1,0,0,1,0,0,1,2,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,3,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,7,0,0,0,0,0,4,0,5,16,0,0,0,0,0,0,1,0,0,0,0,0,0,0,9,0,0


In [12]:
print('Dimension of train_data:',train_df.shape)
print('Dimension of test_data:',test_df.shape)

Dimension of train_data: (61878, 95)
Dimension of test_data: (144368, 94)


In [13]:
train_df.isnull().sum().sum()

0

In [14]:
test_df.isnull().sum().sum()

0

In [15]:
classes=set(train_df.target)
len(classes)

9

In [16]:
print('No of samples across each category:',pd.crosstab(train_df.target,1))


No of samples across each category: col_0        1
target        
Class_1   1929
Class_2  16122
Class_3   8004
Class_4   2691
Class_5   2739
Class_6  14135
Class_7   2839
Class_8   8464
Class_9   4955


In [0]:
train=train_df.copy()
test=test_df.copy()

In [18]:
train['target'] = train['target'].str.replace('Class_', '')
train.target.head()
#target name 'Class_0' to 0.

0    1
1    1
2    1
3    1
4    1
Name: target, dtype: object

In [19]:
# Convertig string target variable into numeric one
train['target'] = pd.to_numeric(train['target'], errors='coerce')
train['target'].head()

0    1
1    1
2    1
3    1
4    1
Name: target, dtype: int64

Defining input and output variable

In [0]:
labels = train["target"].copy()
train = train.drop(['id','target'],axis=1)
test = test.drop('id',axis=1)

**Train and test split**

In [0]:
train,validation,train_labels,validation_labels = train_test_split(train,labels,test_size=0.2,random_state=42)

**Let's start building models**

In [22]:
sgd_clf = SGDClassifier(random_state=42, tol = 0.001, max_iter = 40000,loss="log")
sgd_clf.fit(train,train_labels)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='log',
              max_iter=40000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=42, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [0]:
def evaluate(labelsTrue,predictions1,predictions2):
    if len(predictions1)>0:
        f1 = f1_score(labelsTrue,predictions1,average="weighted")
        print("F1 score: ",f1)
    if len(predictions2)>0:
        logloss = log_loss(labelsTrue,predictions2, eps=1e-15, normalize=True, sample_weight=None, labels=None)
        print("Log loss for predicted probabilities:",logloss)

In [24]:
sgd_prediction1 = sgd_clf.predict(validation)
sgd_prediction2 = sgd_clf.predict_proba(validation)
evaluate(validation_labels,sgd_prediction1,sgd_prediction2)

F1 score:  0.7373488687531389
Log loss for predicted probabilities: 0.7037662849382124


### KNN

In [25]:
knn_clf = KNeighborsClassifier()
knn_clf.fit(train,train_labels)
knn_prediction1 = knn_clf.predict(validation)
knn_prediction2 = knn_clf.predict_proba(validation)
evaluate(validation_labels,knn_prediction1,knn_prediction2)

F1 score:  0.7739562191425366
Log loss for predicted probabilities: 2.3086776076579794


### Random Forest Classifier

In [26]:
rf_clf = RandomForestClassifier(n_estimators=100,n_jobs=-1)
rf_clf.fit(train,train_labels)
rf_prediction1 = rf_clf.predict(validation)
rf_prediction2 = rf_clf.predict_proba(validation)
evaluate(validation_labels,rf_prediction1,rf_prediction2)

F1 score:  0.8009064661509012
Log loss for predicted probabilities: 0.6116598835603284


### Extra Tree Classifier

In [27]:
trees_clf = ExtraTreesClassifier(n_estimators=50,n_jobs=-1)
trees_clf.fit(train,train_labels)
prediction1_trees = trees_clf.predict(validation)
prediction2_trees = trees_clf.predict_proba(validation)
evaluate(validation_labels,prediction1_trees,prediction2_trees)

F1 score:  0.7986324308609025
Log loss for predicted probabilities: 0.6671391740884464


### Multilayer Perceptron Classifier

In [28]:
mlp_clf = MLPClassifier(max_iter = 500)
mlp_clf.fit(train,train_labels)
prediction1_mlp = mlp_clf.predict(validation)
prediction2_mlp = mlp_clf.predict_proba(validation)
evaluate(validation_labels,prediction1_mlp,prediction2_mlp)

F1 score:  0.7810735026561164
Log loss for predicted probabilities: 0.8925115149222539


### AdaBoost

In [29]:
ada_clf = AdaBoostClassifier()
ada_clf.fit(train,train_labels)
prediction1_ada = ada_clf.predict(validation)
prediction2_ada = ada_clf.predict_proba(validation)
evaluate(validation_labels,prediction1_ada,prediction2_ada)

F1 score:  0.6744072921883629
Log loss for predicted probabilities: 2.0261936531308335


### Xgboost Classifier

In [30]:
xgb_clf = XGBClassifier(num_class = 9,objective="multi:softprob",eval_metric="mlogloss")
xgb_clf.fit(train,train_labels)
prediction1_xgb = xgb_clf.predict(validation)
prediction2_xgb = xgb_clf.predict_proba(validation)
evaluate(validation_labels,prediction1_xgb,prediction2_xgb)

F1 score:  0.7578247463382258
Log loss for predicted probabilities: 0.6483268002190926


In [31]:
# finding optimal parameters using grid search

xgb_gs_clf = XGBClassifier(num_class = 9,
                           objective="multi:softprob",
                           eval_metric="mlogloss",
                           seed=42)                         
                        
param_grid = {"max_depth": [10,15],
              "n_estimators": [300,400] , 
              "gamma": [0.03,0.05], 
              "learning_rate": [0.08],
              "min_child_weight": [5,10], 
              "colsample_bytree": [0.4,0.8], 
              "subsample": [0.85]} 

grid_search = GridSearchCV(xgb_gs_clf, 
                           param_grid=param_grid,
                           cv = 2,
                           n_jobs=-1,
                           scoring='neg_log_loss',
                           verbose=2)
grid_search.fit(train,train_labels)

Fitting 2 folds for each of 32 candidates, totalling 64 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed: 165.0min
[Parallel(n_jobs=-1)]: Done  64 out of  64 | elapsed: 356.1min finished


GridSearchCV(cv=2, error_score='raise-deprecating',
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, eval_metric='mlogloss',
                                     gamma=0, learning_rate=0.1,
                                     max_delta_step=0, max_depth=3,
                                     min_child_weight=1, missing=None,
                                     n_estimators=100, n_jobs=1, nthread=None,
                                     num_class=9, objective='multi:softprob...
                                     scale_pos_weight=1, seed=42, silent=None,
                                     subsample=1, verbosity=1),
             iid='warn', n_jobs=-1,
             param_grid={'colsample_bytree': [0.4, 0.8], 'gamma': [0.03, 0.05],
                         'learning_rate': [0.08], 'max_depth': [10, 15],
                        

In [32]:
print(grid_search.best_params_)

{'colsample_bytree': 0.4, 'gamma': 0.05, 'learning_rate': 0.08, 'max_depth': 10, 'min_child_weight': 5, 'n_estimators': 400, 'subsample': 0.85}


In [33]:
grid_prediction1 = grid_search.predict(validation)
grid_prediction2 = grid_search.predict_proba(validation)
evaluate(validation_labels,grid_prediction1,grid_prediction2)

F1 score:  0.8236450346477324
Log loss for predicted probabilities: 0.44959403125885317


In [0]:
xgb_clf = grid_search.best_estimator_
xgb_prediction1 = grid_prediction1
xgb_prediction2 = grid_prediction2

In [36]:
predictions = pd.DataFrame( {'RandomForest': rf_prediction1,
                                  'xgboost': xgb_prediction1,
                                  'NearestNeighbours': knn_prediction1,
                                  'NeuralNetworrk': prediction1_mlp,
                                  'ExtraTrees': prediction1_trees,
                                  'AdaBoost': prediction1_ada,
                                  'SGD':sgd_prediction1,
                        
    })
predictions.head()

,RandomForest,xgboost,NearestNeighbours,NeuralNetworrk,ExtraTrees,AdaBoost,SGD
0,2,7,2,7,2,2,2
1,7,7,7,7,7,1,7
2,6,6,6,6,6,6,6
3,6,6,6,4,6,4,4
4,6,6,6,6,6,6,6


### Build Stacking Model

In [0]:
base1_clf = mlp_clf
base2_clf = rf_clf
base3_clf = knn_clf
base4_clf = ada_clf
base5_clf = sgd_clf
final_clf = xgb_clf

In [0]:
n_folds = 5
n_class = len(classes)
kf = KFold(n_splits= n_folds, shuffle=True, random_state=42)

In [0]:
def get_oof(clf, x_train, y_train, x_test):
    ntest = x_test.shape[0]
    oof_train = np.zeros((x_train.shape[0],n_class))
    oof_test = np.zeros((ntest,n_class))
    oof_test_temp = np.empty((n_folds, ntest))
   
    for i,(train_index, test_index) in enumerate(kf.split(x_train)):
        
        x_tr = x_train[train_index]
        y_tr = y_train[train_index]
        x_te = x_train[test_index]
  
        
        clf.fit(x_tr, y_tr)

        pred_te = clf.predict_proba(x_te)
        oof_train[test_index,:] = pred_te
        
        pred_test = clf.predict_proba(x_test)
        oof_test += pred_test

    return oof_train, oof_test/n_folds

In [0]:
x_train = train.values
test_data = test.values
y_train = train_labels.ravel()

In [0]:
base1_oof_train, base1_oof_test = get_oof(base1_clf, x_train, y_train, test_data)
base2_oof_train, base2_oof_test = get_oof(base2_clf, x_train, y_train, test_data)
base3_oof_train, base3_oof_test = get_oof(base3_clf, x_train, y_train, test_data)
base4_oof_train, base4_oof_test = get_oof(base4_clf, x_train, y_train, test_data) 
base5_oof_train, base5_oof_test = get_oof(base5_clf, x_train, y_train, test_data)

In [0]:
x_train = np.concatenate((base1_oof_train, 
                          base2_oof_train,
                          base3_oof_train,
                          base4_oof_train,
                          base5_oof_train), axis=1)
test_data = np.concatenate((base1_oof_test,
                         base2_oof_test,
                         base3_oof_test,
                         base4_oof_test,
                         base5_oof_test), axis=1)

In [43]:
final_clf.fit(x_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.4, eval_metric='mlogloss',
              gamma=0.05, learning_rate=0.08, max_delta_step=0, max_depth=10,
              min_child_weight=5, missing=None, n_estimators=400, n_jobs=1,
              nthread=None, num_class=9, objective='multi:softprob',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              seed=42, silent=None, subsample=0.85, verbosity=1)

In [0]:
prediction1_stacked = final_clf.predict(test_data)
prediction2_stacked = final_clf.predict_proba(test_data)
#evaluate(validation_labels,prediction1_stacked,prediction2_stacked)

In [60]:
len(prediction1_stacked)

144368

In [90]:
submission = pd.DataFrame()
submission['id'] = test_df['id']
submission['class'] = prediction1_stacked
submission.head()

,id,class
0,1,4
1,2,8
2,3,6
3,4,3
4,5,9


In [0]:
#final submission is needed in one hot encoded form.
submission['class'] = submission['class'].astype('category')

submission = pd.get_dummies(submission)

In [96]:
submission.head()

,id,class_1,class_2,class_3,class_4,class_5,class_6,class_7,class_8,class_9
0,1,0,0,0,1,0,0,0,0,0
1,2,0,0,0,0,0,0,0,1,0
2,3,0,0,0,0,0,1,0,0,0
3,4,0,0,1,0,0,0,0,0,0
4,5,0,0,0,0,0,0,0,0,1


In [0]:
submission.to_csv('HotEncodedSubmission1.csv',index = False)


In [101]:
submission.shape

(144368, 10)